In [1]:
import helper_data_load as helper_load
import os
import helper_prediction_class as helper
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression, mutual_info_classif, f_classif
from sklearn.metrics import accuracy_score
from sklearn import metrics
from lifelines.utils import concordance_index
import numpy as np


In [2]:
feat_sel_method = "f_classif"
nr_sol_temp = "-1"

In [3]:
data_norm_chum, data_norm_iucpq = helper_load.get_norm_data(local=True, classification=True)
random_state=42
nr_sol = -1
data_norm_iucpq.shape

223 patients with clinical and radiation data for CHUM.
162 patients with clinical and radiation data for IUCPQ.
149 patients with clinical and radiation data for CHUM and classification.
108 patients with clinical and radiation data for IUCPQ and classification.


/Users/iucpq/Documents/IUCPQ/IUCPQ/Predictions/helper_data_load.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_norm.columns = data_norm.columns.str.replace('.','_')
/Users/iucpq/Documents/IUCPQ/IUCPQ/Predictions/helper_data_load.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_norm.columns = data_norm.columns.str.replace('.','_')
/Users/iucpq/Documents/IUCPQ/IUCPQ/Predictions/helper_data_load.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["class_os_days"][df.class

(108, 852)

In [4]:
# SVC
df_result_SVC= pd.DataFrame(columns=["nr_feat","c_idx_svc"])
rf_model = SVC()
nr_feat = 3
nr_solutions=5
k_fold_nr = 5
c_idxs=[]
accs=[]
aucs=[]
df_evals = pd.DataFrame(columns=["nr_feat", "mean_auc", "mean_acc", "c_idx", "c_concat"])
for nr_feat in range(1,10):
    # k fold:
    cv = KFold(n_splits=k_fold_nr, random_state=random_state, shuffle=True)
    y_test_cv = []
    y_pred_cv = []
    print("whole dataset shape: ")
    print(data_norm_iucpq.shape)

    for train_index, test_index in cv.split(data_norm_iucpq):
        # get train and test sets for this fold
        X_train, X_test = data_norm_iucpq.loc[train_index, data_norm_iucpq.columns != 'class_os_days'], data_norm_iucpq.loc[test_index, data_norm_iucpq.columns != 'class_os_days']
        y_train, y_test = data_norm_iucpq.loc[train_index, "class_os_days"], data_norm_iucpq.loc[test_index, "class_os_days"]
        print("x_train: ", X_train.shape)
        print("y_train: ", y_train.shape)
        print("X_test: ", X_test.shape)
        print("y_test: ", y_test.shape)
        y_train = y_train.astype('int')
        y_test = y_test.astype('int')
        # calculate most correlating features of that fold
        ## select feats: model_str, y_predict = helper.select_feats(feat_sel_method, X_train, y_train, X_test, rf_model, nr_feat, nr_solutions, random_state)
        if feat_sel_method in  ["f_classif"]:
            if feat_sel_method == "f_classif":
                score_func = f_classif
            ## fit_apply_kbest:  X_train_new, X_test_new = fit_apply_kbest(X_train, y_train, score_func, nr_feat, X_test)
            data_transformer = SelectKBest(score_func=score_func, k=nr_feat).fit(X_train, y_train)
            X_train_new = data_transformer.transform(X_train)
            X_test_new = data_transformer.transform(X_test)
            feat_names = data_transformer.get_feature_names_out
            mask = data_transformer.get_support() #list of booleans
            new_features = [] # The list of your K best features
            for bool, feature in zip(mask, X_train.columns):
                if bool:
                    new_features.append(feature)
            print(new_features)
            print(feat_names)
            print("X train after feat selec: ", X_train_new.shape)
            print("X test after feat selec: ", X_test_new.shape)

            feat_to_use = []
            ## fit and predict: model_str, y_predict = fit_and_predict(model, X_train_new, y_train, feat_to_use, X_test_new, random_state, feat_sel_method)
            print("in fit_and_predict: ", type(rf_model))
            if feat_sel_method not in [ "f_classif"]:
                X_train = X_train[feat_to_use]
                X_test = X_test[feat_to_use]
            print(str(rf_model))
            model_str = str(rf_model)
            if str(rf_model) in ["SVC()", "RandomForestClassifier(random_state=42)", "BaggingClassifier(random_state=42)", "GradientBoostingClassifier(random_state=42)", "DecisionTreeClassifier(random_state=42)", "KNeighborsClassifier()"]:
                if str(rf_model) == "SVC()":
                    random_grid = {'kernel': ['linear', 'rbf','poly', 'sigmoid'], 
                                'C':[1.5, 10],
                                'gamma': [1e-7, 1e-4]
                                }
            model = RandomizedSearchCV(estimator = rf_model, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=random_state, n_jobs = -1)
        print("in fit_right before error: ", type(model))
        print(len(X_train_new))
        model.fit(X_train_new, y_train)
        y_predict = model.predict(X_test_new)
        print(y_test)
        print(y_predict)
        c_idx = concordance_index(y_test, y_predict)
        c_idxs.append(c_idx)
        print("c: ", c_idx)
        print(c_idxs)
        acc = accuracy_score(y_test, y_predict)
        accs.append(acc)
        print("acc: ", acc)
        print(accs)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict, pos_label=2)
        auc = metrics.auc(fpr, tpr)
        aucs.append(auc)
        print("auc: ", auc)
        print(aucs)

        y_test_cv.append(y_test)
        y_pred_cv.append(y_predict)

    mean_acc = np.mean(accs)
    print(mean_acc)
    mean_c_idx = np.mean(c_idxs)
    print(mean_c_idx)
    y_test_cv = [j for sub in y_test_cv for j in sub]
    y_pred_cv = [j for sub in y_pred_cv for j in sub]
    c_concat = concordance_index(y_test_cv, y_pred_cv)
    print(c_concat)
    auc_mean = np.mean(aucs)
    print(auc_mean)
    df_temp = pd.DataFrame(dict({"nr_feat":nr_feat, "mean_auc": auc_mean, "mean_acc": mean_acc, "c_idx": mean_c_idx, "c_concat": c_concat}), index=[0])
    display(df_temp)
    df_evals = pd.concat([df_evals, df_temp])

#df_result_SVC = helper.not_ensemble_methods(rf_model, df_result_SVC, data_norm_iucpq, random_state=random_state, feat_sel_meth=feat_sel_method, nr_sol=nr_sol)
print("Trained SVC")
os.system("echo Trained SVC")
df_evals
#result_discovery_IUCPQ_all = helper.all_methods(data_norm_iucpq, random_state, FEAT_SEL_METH=feat_sel_method, NR_SOL=nr_sol)



whole dataset shape: 
(108, 852)
x_train:  (86, 851)
y_train:  (86,)
X_test:  (22, 851)
y_test:  (22,)
['wavelet_HHL_glrlm_RunVariance']
<bound method SelectorMixin.get_feature_names_out of SelectKBest(k=1)>
X train after feat selec:  (86, 1)
X test after feat selec:  (22, 1)
in fit_and_predict:  <class 'sklearn.svm._classes.SVC'>
SVC()
in fit_right before error:  <class 'sklearn.model_selection._search.RandomizedSearchCV'>
86
Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/iucpq/Documents/IUCPQ/Environments/venv1/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 16 is smaller than n_iter=50. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END .....................C=1.5, gamma=1e-07, kernel=rbf; total time=   0.0s
[CV] END .....................C=1.5, gamma=1e-07, kernel=rbf; total time=   0.0s
[CV] END .....................C=1.5, gamma=1e-07, kernel=rbf; total time=   0.0s
[CV] END ....................C=1.5, gamma=1e-07, kernel=poly; total time=   0.0s
[CV] END ....................C=1.5, gamma=1e-07, kernel=poly; total time=   0.0s
[CV] END .................C=1.5, gamma=1e-07, kernel=sigmoid; total time=   0.0s
[CV] END ....................C=1.5, gamma=1e-07, kernel=poly; total time=   0.0s
[CV] END .................C=1.5, gamma=1e-07, kernel=sigmoid; total time=   0.0s
[CV] END .................C=1.5, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .................C=

/Users/iucpq/Documents/IUCPQ/Environments/venv1/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 16 is smaller than n_iter=50. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/iucpq/Documents/IUCPQ/Environments/venv1/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 16 is smaller than n_iter=50. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END ....................C=1.5, gamma=1e-07, kernel=poly; total time=   0.0s
[CV] END .....................C=1.5, gamma=1e-07, kernel=rbf; total time=   0.0s
[CV] END ....................C=1.5, gamma=1e-07, kernel=poly; total time=   0.0s
[CV] END .....................C=1.5, gamma=1e-07, kernel=rbf; total time=   0.0s
[CV] END .................C=1.5, gamma=1e-07, kernel=sigmoid; total time=   0.0s
[CV] END .................C=1.5, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .................C=1.5, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END ....................C=1.5, gamma=1e-07, kernel=poly; total time=   0.1s
[CV] END .................C=1.5, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END ...................

...

['wavelet_HLL_glrlm_RunLengthNonUniformityNormalized', 'wavelet_HLH_glrlm_RunLengthNonUniformityNormalized', 'wavelet_HLH_glrlm_RunPercentage', 'wavelet_HHL_gldm_LargeDependenceEmphasis', 'wavelet_HHH_glrlm_LongRunEmphasis', 'wavelet_HHH_glrlm_RunPercentage', 'wavelet_HHH_glrlm_RunVariance', 'wavelet_HHH_glrlm_ShortRunEmphasis', 'wavelet_HHH_gldm_LargeDependenceEmphasis']
<bound method SelectorMixin.get_feature_names_out of SelectKBest(k=9)>
X train after feat selec:  (87, 9)
X test after feat selec:  (21, 9)
in fit_and_predict:  <class 'sklearn.svm._classes.SVC'>
SVC()
in fit_right before error:  <class 'sklearn.model_selection._search.RandomizedSearchCV'>
87
Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END ..................C=1.5, gamma=1e-07, kernel=linear; total time=   0.0s
[CV] END .....................C=1.5, gamma=1e-07, kernel=rbf; total time=   0.0s[CV] END ..................C

/Users/iucpq/Documents/IUCPQ/Environments/venv1/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 16 is smaller than n_iter=50. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


,nr_feat,mean_auc,mean_acc,c_idx,c_concat
0,9,0.6367,0.627994,0.6367,0.62037


Trained SVC
Trained SVC


,nr_feat,mean_auc,mean_acc,c_idx,c_concat
0,1,0.662165,0.656277,0.662165,0.657407
0,2,0.671748,0.665584,0.671748,0.675926
0,3,0.665736,0.659596,0.665736,0.648148
0,4,0.654692,0.647403,0.654692,0.611111
0,5,0.648065,0.640087,0.648065,0.611111
0,6,0.645731,0.636724,0.645731,0.620370
0,7,0.642301,0.632962,0.642301,0.611111
0,8,0.638166,0.629004,0.638166,0.601852
0,9,0.636700,0.627994,0.636700,0.620370
